# Hourglass Network
## For keypoints extraction

In [ ]:
import os
import cv2
import numpy as np
from typing import List, Tuple
from hourglass import HourglassNetwork

# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"


### Path to data

In [ ]:
IMAGES_PATH = "../YCB-Video_data/data/0010"
IMAGES = [f"{IMAGES_PATH}/{f}" for f in os.listdir(IMAGES_PATH) if f.endswith("color.png")]
DIM = (480, 640, 3)
N = len(IMAGES)

KEYPOINTS_PATH = "../YCB-Video_data/keypoints/0010_gt_keypoints2d.npy"
KEYPOINTS = np.load(KEYPOINTS_PATH)

### Keypoints to heatmaps finctions

In [ ]:
def gaussian_k(x0: int, y0: int, sigma: float, height: int, width: int) -> np.ndarray:
        """ Make a square gaussian kernel centered at (x0, y0) with sigma as SD. """
        x = np.arange(0, width, 1, float) ## (width,)
        y = np.arange(0, height, 1, float)[:, np.newaxis] ## (height,1)
        return np.exp(-((x-x0)**2 + (y-y0)**2) / (2*sigma**2))

def generate_heatmap(height: int, width: int, landmarks: List[Tuple[int, int]], s: int = 3) -> np.ndarray:
        """ Generate a full Heap Map for every landmarks in an array
        Args:
            height    : The height of Heat Map (the height of target output)
            width     : The width  of Heat Map (the width of target output)
            joints    : [(x1,y1),(x2,y2)...] containing landmarks
            maxlenght : Lenght of the Bounding Box
        """
        Nlandmarks = len(landmarks)
        hm = np.zeros((height, width, Nlandmarks), dtype = np.float32)
        for i in range(Nlandmarks):
            if not np.array_equal(landmarks[i], [-1,-1]):
             
                hm[:,:,i] = gaussian_k(landmarks[i][0],
                                        landmarks[i][1],
                                        s,height, width)
            else:
                hm[:,:,i] = np.zeros((height,width))
        return hm

### Batch generator

In [ ]:
def batchgen(images: List[str], keypoints: np.ndarray, dataset_size: int, batch_size: int, epochs: int) -> Tuple[np.ndarray, np.ndarray]:
    for _ in range(epochs):
        for i in range(0, dataset_size, batch_size):
            x_batch = np.array([cv2.imread(img) for img in images[i:i+batch_size]])
            y_batch = np.array([generate_heatmap(DIM[0]//4, DIM[1]//4, k//4) for k in keypoints[i:i+batch_size]])
            yield x_batch, y_batch


### Create Hourglass neural network

In [ ]:
num_classes = 9
num_stacks = 2
num_filters = 32

net = HourglassNetwork(num_classes, num_stacks,
                       num_filters, DIM, (120, 160))
# net.summary()

### Train

In [ ]:
batch_size = 4
epochs = 50
hist = net.fit(data_generator=batchgen(IMAGES, KEYPOINTS, dataset_size=N, batch_size=batch_size, epochs=epochs),
        dataset_size=N,
        batch_size=batch_size,
        epochs=epochs)


In [ ]:
def overlay_heatmap(img: np.ndarray, heatmap: np.ndarray) -> np.ndarray:
    _heatmap = heatmap.sum(axis=-1) # sum all heatmaps
    _heatmap = cv2.normalize(_heatmap, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    _heatmap = cv2.resize(_heatmap, (640, 480))  # resize
    _heatmap = cv2.applyColorMap(_heatmap, cv2.COLORMAP_JET)
    return cv2.addWeighted(_heatmap, .7, img, .3, 0)


gen = batchgen(IMAGES, KEYPOINTS, dataset_size=N, batch_size=1, epochs=1)
x, y = next(gen)
out = net.predict(x)
overlay = overlay_heatmap(x[0], out[0])
cv2.imshow("Heatmap overlay", overlay)
cv2.waitKey()
cv2.destroyAllWindows()